In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from tqdm import tqdm
import h5py 
import os
import sys
import glob
sys.path.append('../data_generation/')
os.environ['CUDA_VISIBLE_DEVICES']='3'
import utils

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True)
model = AutoModel.from_pretrained("zhihan1996/DNABERT-2-117M", trust_remote_code=True).to('cuda')
model.eval();

/home/ztang/.conda/envs/torch_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/home/ztang/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-2-117M/1d020b803b871a976f5f3d5565f0eac8f2c7bb81/bert_layers.py:125: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertM

In [2]:
datalen = '998'
file = h5py.File("../data/CAGI/"+datalen+"/CAGI_onehot.h5", "r")
alt = file['alt']
ref = file['ref']

In [3]:
N, L, A = alt.shape
batch_size = 50
cos = []
dot = []
l1 = []
l2 = []
for i in tqdm(range(0,N,batch_size)):
    b_size = batch_size
    if i + batch_size > N:
        b_size = N-i
    onehot = np.concatenate((ref[i:i+b_size],alt[i:i+b_size]))
    seq = utils.onehot_to_seq(onehot)
    with torch.inference_mode():
        input_ids = tokenizer(seq, return_tensors="pt",padding='max_length',max_length=226)["input_ids"].to('cuda')
        hidden_states = model(input_ids)[0].cpu().detach().numpy()
    for a in range(b_size):
        ref_out = hidden_states[a]
        alt_out = hidden_states[a+b_size]
        cos.append((ref_out * alt_out).sum()/(np.linalg.norm(ref_out)*np.linalg.norm(alt_out)))
        dot.append((ref_out * alt_out).sum())
        l1.append(np.absolute(ref_out - alt_out).sum())
        l2.append(np.square(ref_out - alt_out).sum())

100%|██████████| 369/369 [05:13<00:00,  1.18it/s]


In [4]:
output = h5py.File('../data/CAGI/'+'cagi_'+datalen+'_'+'dnabert2.h5', 'w')
output.create_dataset('cosine', data=np.array(cos))
output.create_dataset('dot', data=np.array(dot))
output.create_dataset('l1', data=np.array(l1))
output.create_dataset('l2', data=np.array(l2))
output.close()

In [5]:
import h5py
import pandas as pd
import numpy as np
from operator import itemgetter
import seaborn as sns
import matplotlib.pyplot as plt 
import scipy.stats as stats
datalen = '998'
cagi_df = pd.read_csv('../data/CAGI/'+datalen+'/final_cagi_metadata.csv',
                      index_col=0).reset_index()
target = cagi_df['6'].values.tolist()
exp_list = cagi_df['8'].unique()
cagi_result = h5py.File('../data/CAGI/'+'cagi_'+datalen+'_'+'dnabert2.h5', 'r')

In [6]:
perf = []
for key in cagi_result.keys():
    print(key)
    cagi_llr = cagi_result[key]
    for exp in ['LDLR','SORT1','F9','PKLR']:
        sub_df = cagi_df[cagi_df['8'] == exp]
        exp_target = np.array(target)[sub_df.index.to_list()]
        exp_pred = np.squeeze(cagi_llr)[sub_df.index.to_list()]
        exp_target = np.absolute(exp_target)
        exp_pred = exp_pred
        print(exp)
        perf.append(stats.pearsonr(exp_pred,exp_target)[0])
        print(stats.pearsonr(exp_pred,exp_target)[0])

cosine
LDLR
-0.08258992666052817
SORT1
-0.010052575379520095
F9
0.027784013416209896
PKLR
0.013793714344828479
dot
LDLR
-0.08532688469556723
SORT1
-0.01534769512394398
F9
-0.0077194387091766075
PKLR
0.30991893844919394
l1
LDLR
0.07815354726339352
SORT1
0.009764958350297075
F9
-0.020930475924285253
PKLR
0.0014890268020691902
l2
LDLR
0.07581393697686814
SORT1
0.009826799329970284
F9
-0.02904052790222869
PKLR
0.0008220143192249419
